**<h1><center>Georeferencing CMIP6 data and extracting an area of interest (AOI)</h1></center>**<br>
This script was developed on Python 3.9.19.<br>
Ensure that the following packages are installed:  *pandas, xarray, numpy*<br>
Feel free to optimise this script and share the update<br>

**1. Importing libraries**

In [1]:
import xarray as xr
import numpy as np

**2. Creating a function to for georeferencing and extracting the AOI**

In [ ]:

def georef_clip (input_file, aoi, output_file): 
    dataset = xr.open_dataset(input_file) # Opening the CMIP6 dataset

    # Georeferencing step
    dataset["lon"] = np.where(dataset["lon"] > 180, dataset["lon"] - 360, dataset["lon"]) # Changing the 0-360 longitude format to use the -180 to 180 format
    dataset = dataset.sortby(dataset.lon) # Sorting the data by longitude values

    # Extracting the area of interest (AOI)

        # Finding the coordinates in the dataset that is closest to the provided AOI. 
    lat_max = np.argmin(np.abs(dataset["lat"].values - aoi[0]))
    lat_min = np.argmin(np.abs(dataset["lat"].values - aoi[1]))
    lon_max = np.argmin(np.abs(dataset["lon"].values - aoi[2]))
    lon_min = np.argmin(np.abs(dataset["lon"].values - aoi[3]))

        # Getting the id of the closest coordinates
    lon_min_id = dataset["lon"].values[lon_min]
    lon_max_id = dataset["lon"].values[lon_max]
    lat_min_id = dataset["lat"].values[lat_min]
    lat_max_id = dataset["lat"].values[lat_max]

        # Clipping the data using the identified coordinates in the dataset that close to the provided AOI
    clipped_data = dataset.sel(lon=slice(lon_min_id, lon_max_id), lat=slice(lat_min_id, lat_max_id))

    # Saving the result
    clipped_data.to_netcdf(output_file)

    print (f"Process completed. Data saved here {output_file}")


**3. Georeferencing and extracting the AOI**

In [ ]:
# Setting the parameters to be used by the function
input_file = "C:/Users/ilung/Documents/uas_day_IPSL-CM6A-LR_ssp245_r1i1p1f1_gr_20150101-21001231.nc" # input CMIP6 file
aoi = [15, -3, 3, -14] # The AOI bounding box. Must use the following format [lat_max, lat_min, lon_max, lon_min]
output_file = "C:/Users/ilung/Documents/uas_day_clip.nc" # Output file

# Launching the function to georeference and extract the AOI
georef_clip(input_file, aoi, output_file)